In [1]:
import argparse
import os
import shutil
from src.analysis.project_parser import parse_project
from src.analysis.dependency_resolver import resolve_dependencies
from src.analysis.feature_mapper import map_features_to_components
from src.generation.project_generator import generate_project
from src.generation.feature_integration import integrate_features
from src.models.llm_inference import query_llm
from src.models.prompt_templates import get_prompt_template, get_prompt_template_feature
from src.utils.logger import logger
from src.utils.file_operations import read_file, write_file
from src.vector_database.db_query import query_vector_database

In [10]:
from src.vector_database.db_load import load_vector_database

In [2]:
logger.info("Starting the Factory Feature program")
# Define paths
old_project_path = "project_old"
new_project_path = "project_new"

In [3]:
# Step 1: Parse the old project
logger.info("Parsing the project structure")
project_data = parse_project(old_project_path)

In [4]:
project_data

[{'path': 'project_old/app.py',
  'content': 'from utils.helpers import greet\n\ndef main():\n    name = input("Enter your name: ")\n    print(greet(name))\n\nif __name__ == "__main__":\n    main()'},
 {'path': 'project_old/requirements.txt',
  'content': '# Python dependencies\nflask'},
 {'path': 'project_old/utils/helpers.py',
  'content': 'def greet(name):\n    """\n    Returns a greeting message for the provided name.\n\n    Args:\n        name: The name of the person to greet.\n\n    Returns:\n        A greeting string.\n    """\n    return f"Hello, {name}! Welcome to the project."'}]

In [5]:
type(project_data)

list

In [6]:
from src.vector_database.db_builder_simple import build_vector_database_simple


In [7]:
vector_db = build_vector_database_simple(project_data)
logger.info("Vector database simple created for the project")

/mnt/c/Blog/Factory-Feature/src/vector_database/db_builder_simple.py:16: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/mnt/c/Blog/Building-LLM-from-Scratch-in-Python/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
vector_db

In [6]:
from src.vector_database.db_builder import build_vector_database


In [7]:
# With persistence
vector_db = build_vector_database(project_data, persist_directory="./chroma_db")

/mnt/c/Blog/Factory-Feature/src/vector_database/db_builder.py:26: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/mnt/c/Blog/Building-LLM-from-Scratch-in-Python/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mnt/c/Blog/Factory-Feature/src/vector_database/db_builder.py:35: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatic

In [11]:
vector_db = load_vector_database(persist_directory="./chroma_db")

/mnt/c/Blog/Factory-Feature/src/vector_database/db_load.py:63: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_db = Chroma(


In [12]:
vector_db

In [13]:
from src.vector_database.display_content import display_first_five_documents
# Display the first 5 documents in the vector database
display_first_five_documents(vector_db)

Displaying the first 3 documents:
Document ID: 6434b73e-f406-4852-8ff9-a9a3d252c8fc
Content:
Path: project_old/app.py
Content:
from utils.helpers import greet

def main():
    name = input("Enter your name: ")
    print(greet(name))

if __name__ == "__main__":
    main()
Metadata: {'source': 'project_old/app.py'}
--------------------------------------------------------------------------------
Document ID: adeca2f7-04af-4ce6-b30f-2041cebb7270
Content:
Path: project_old/requirements.txt
Content:
# Python dependencies
flask
Metadata: {'source': 'project_old/requirements.txt'}
--------------------------------------------------------------------------------
Document ID: b70470ba-b70f-466e-aa3e-1ed31c73faf3
Content:
Path: project_old/utils/helpers.py
Content:
def greet(name):
    """
    Returns a greeting message for the provided name.

    Args:
        name: The name of the person to greet.

    Returns:
        A greeting string.
    """
    return f"Hello, {name}! Welcome to the project

In [15]:
# Step 2: Resolve dependencies
logger.info("Resolving project dependencies")
dependencies = resolve_dependencies(old_project_path)
logger.info(f"Dependencies found: {dependencies.keys()}")

In [16]:
# Step 3: Map features to project components
logger.info("Mapping feature request to project components")
feature_template = get_prompt_template_feature()
project_context = "\n".join([f"{key}: {value}" for key, value in dependencies.items()])


In [17]:
project_context

"requirements.txt: ['# Python dependencies\\n', 'flask']"

In [18]:
from src.analysis.tree import get_tree

if __name__ == "__main__":
    old_project_path = "project_old"
    try:
        # Get the tree structure without content
        tree = get_tree(old_project_path)
        print(tree)
        
        # Get the tree structure with content
        tree_with_content = get_tree(old_project_path, content=True)
        print(tree_with_content)
    except FileNotFoundError as e:
        print(e)


{'project_name': 'project_old', 'structure': [{'type': 'file', 'name': 'app.py', 'path': 'project_old/app.py'}, {'type': 'file', 'name': 'requirements.txt', 'path': 'project_old/requirements.txt'}, {'type': 'directory', 'name': 'utils', 'path': 'project_old/utils', 'children': [{'type': 'file', 'name': 'helpers.py', 'path': 'project_old/utils/helpers.py'}]}]}
{'project_name': 'project_old', 'structure': [{'type': 'file', 'name': 'app.py', 'path': 'project_old/app.py', 'content': 'from utils.helpers import greet\n\ndef main():\n    name = input("Enter your name: ")\n    print(greet(name))\n\nif __name__ == "__main__":\n    main()'}, {'type': 'file', 'name': 'requirements.txt', 'path': 'project_old/requirements.txt', 'content': '# Python dependencies\nflask'}, {'type': 'directory', 'name': 'utils', 'path': 'project_old/utils', 'children': [{'type': 'file', 'name': 'helpers.py', 'path': 'project_old/utils/helpers.py', 'content': 'def greet(name):\n    """\n    Returns a greeting message f

In [19]:
tree = get_tree(old_project_path)


In [20]:
tree

{'project_name': 'project_old',
 'structure': [{'type': 'file',
   'name': 'app.py',
   'path': 'project_old/app.py'},
  {'type': 'file',
   'name': 'requirements.txt',
   'path': 'project_old/requirements.txt'},
  {'type': 'directory',
   'name': 'utils',
   'path': 'project_old/utils',
   'children': [{'type': 'file',
     'name': 'helpers.py',
     'path': 'project_old/utils/helpers.py'}]}]}

In [21]:
tree_with_content = get_tree(old_project_path, content=True)


In [22]:
tree_with_content

{'project_name': 'project_old',
 'structure': [{'type': 'file',
   'name': 'app.py',
   'path': 'project_old/app.py',
   'content': 'from utils.helpers import greet\n\ndef main():\n    name = input("Enter your name: ")\n    print(greet(name))\n\nif __name__ == "__main__":\n    main()'},
  {'type': 'file',
   'name': 'requirements.txt',
   'path': 'project_old/requirements.txt',
   'content': '# Python dependencies\nflask'},
  {'type': 'directory',
   'name': 'utils',
   'path': 'project_old/utils',
   'children': [{'type': 'file',
     'name': 'helpers.py',
     'path': 'project_old/utils/helpers.py',
     'content': 'def greet(name):\n    """\n    Returns a greeting message for the provided name.\n\n    Args:\n        name: The name of the person to greet.\n\n    Returns:\n        A greeting string.\n    """\n    return f"Hello, {name}! Welcome to the project."'}]}]}

In [23]:
user_request="Add logging functionality to all major modules in the project"

In [24]:
tree

{'project_name': 'project_old',
 'structure': [{'type': 'file',
   'name': 'app.py',
   'path': 'project_old/app.py'},
  {'type': 'file',
   'name': 'requirements.txt',
   'path': 'project_old/requirements.txt'},
  {'type': 'directory',
   'name': 'utils',
   'path': 'project_old/utils',
   'children': [{'type': 'file',
     'name': 'helpers.py',
     'path': 'project_old/utils/helpers.py'}]}]}

In [25]:
tree_string = str(tree) 
print(tree_string)

{'project_name': 'project_old', 'structure': [{'type': 'file', 'name': 'app.py', 'path': 'project_old/app.py'}, {'type': 'file', 'name': 'requirements.txt', 'path': 'project_old/requirements.txt'}, {'type': 'directory', 'name': 'utils', 'path': 'project_old/utils', 'children': [{'type': 'file', 'name': 'helpers.py', 'path': 'project_old/utils/helpers.py'}]}]}


In [26]:
### ATTENTTION WE ARE PARSING ONLY THE TREE WITHOUT THE CONTENT
### Maybe we can parse the full tree with content but we need to think a solution 
## Such the project_context does not supperate the limit of tokens.
## Maybe chunking or another technique.

project_context=project_context+tree_string

In [27]:
project_context

"requirements.txt: ['# Python dependencies\\n', 'flask']{'project_name': 'project_old', 'structure': [{'type': 'file', 'name': 'app.py', 'path': 'project_old/app.py'}, {'type': 'file', 'name': 'requirements.txt', 'path': 'project_old/requirements.txt'}, {'type': 'directory', 'name': 'utils', 'path': 'project_old/utils', 'children': [{'type': 'file', 'name': 'helpers.py', 'path': 'project_old/utils/helpers.py'}]}]}"

In [28]:
#feature_request = feature_template.format(feature_request=user_request, project_context=project_context)


In [29]:
feature_request=user_request

In [29]:
# Step 0: Verification of LLM Connection
from src.models.llm_inference import query_llm
prompt = "Who is man's best friend?"
response = query_llm(prompt)
print(response)

Debug: Received prompt: Who is man's best friend?
Debug: Vector DB provided: False
Debug: Formatted prompt:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
None<|eot_id|><|start_header_id|>user<|end_header_id|>
Who is man's best friend?<|eot_id|><|start_header_id|>assistant<|end_header_id|>



/mnt/c/Blog/Building-LLM-from-Scratch-in-Python/.venv/lib/python3.10/site-packages/ibm_watsonx_ai/foundation_models/utils/utils.py:415: LifecycleWarning: Model 'meta-llama/llama-3-70b-instruct' is in deprecated state from 2024-12-02 until 2025-02-03. IDs of alternative models: None. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warnings.warn(


Debug: Querying without vector database...
Debug: Raw response from model: That's an easy one! Man's best friend is the dog! Dogs have been human companions for thousands of years, providing loyalty, affection, and protection. They are often referred to as "man's best friend" because of their unique ability to form strong bonds with humans and provide comfort, companionship, and assistance.
Debug: Response as string: That's an easy one! Man's best friend is the dog! Dogs have been human companions for thousands of years, providing loyalty, affection, and protection. They are often referred to as "man's best friend" because of their unique ability to form strong bonds with humans and provide comfort, companionship, and assistance.
That's an easy one! Man's best friend is the dog! Dogs have been human companions for thousands of years, providing loyalty, affection, and protection. They are often referred to as "man's best friend" because of their unique ability to form strong bonds with 

In [30]:
response

'That\'s an easy one! Man\'s best friend is the dog! Dogs have been human companions for thousands of years, providing loyalty, affection, and protection. They are often referred to as "man\'s best friend" because of their unique ability to form strong bonds with humans and provide comfort, companionship, and assistance.'

In [30]:
# Step 1: Feature Analysis
logger.info("Performing feature analysis")
feature_nodes_template = get_prompt_template("feature_analysis_nodes")
feature_edges_template = get_prompt_template("feature_analysis_edges")
impact_report_template = get_prompt_template("feature_impact_report")


## Feature Analysis without VectorDB

In [ ]:
#1 feature_nodes

In [31]:
feature_nodes = feature_nodes_template.format(
    feature_request=user_request, project_context=project_context
)

In [31]:
feature_nodes_response = query_llm(user_input=feature_nodes)

Debug: Received prompt: Analyze the project context and identify the nodes (files or components) impacted by the following feature request:

Feature Request: Add logging functionality to all major modules in the project

Project Context: requirements.txt: ['# Python dependencies\n', 'flask']{'project_name': 'project_old', 'structure': [{'type': 'file', 'name': 'app.py', 'path': 'project_old/app.py'}, {'type': 'file', 'name': 'requirements.txt', 'path': 'project_old/requirements.txt'}, {'type': 'directory', 'name': 'utils', 'path': 'project_old/utils', 'children': [{'type': 'file', 'name': 'helpers.py', 'path': 'project_old/utils/helpers.py'}]}]}

Provide a list of impacted nodes and their roles in the project.
Debug: Vector DB provided: False
Debug: Formatted prompt:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
None<|eot_id|><|start_header_id|>user<|end_header_id|>
Analyze the project context and identify the nodes (files or components) impacted by the following feature 

/mnt/c/Blog/Building-LLM-from-Scratch-in-Python/.venv/lib/python3.10/site-packages/ibm_watsonx_ai/foundation_models/utils/utils.py:415: LifecycleWarning: Model 'meta-llama/llama-3-70b-instruct' is in deprecated state from 2024-12-02 until 2025-02-03. IDs of alternative models: None. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warnings.warn(


Debug: Querying without vector database...
Debug: Raw response from model: Based on the project context and the feature request, the impacted nodes are:

1. **app.py**: This is the main application file, and it's likely that logging functionality will need to be added to this file to log events and errors related to the application's core functionality. **Role: Main Application File**

2. **helpers.py**: This file is part of the utils module, and it may contain helper functions that are used throughout the application. Adding logging functionality to this file will help track the usage and behavior of these helper functions. **Role: Utility Functions**

3. **requirements.txt**: Although this file doesn't need to be directly modified to add logging functionality, it may need to be updated to include a logging library (e.g., Python's built-in `logging` module or a third-party library like `loguru`) as a dependency. **Role: Dependency Management**

4. **New logging configuration file (e.g

In [ ]:
#2 feature_edges

In [33]:
feature_edges = feature_edges_template.format(
    feature_request=user_request, project_context=project_context
)

In [35]:
feature_edges_response = query_llm(user_input=feature_edges)

Debug: Received prompt: Analyze the project context and determine the dependencies (edges) impacted by the following feature request:

Feature Request: Add logging functionality to all major modules in the project

Project Context: requirements.txt: ['# Python dependencies\n', 'flask']{'project_name': 'project_old', 'structure': [{'type': 'file', 'name': 'app.py', 'path': 'project_old/app.py'}, {'type': 'file', 'name': 'requirements.txt', 'path': 'project_old/requirements.txt'}, {'type': 'directory', 'name': 'utils', 'path': 'project_old/utils', 'children': [{'type': 'file', 'name': 'helpers.py', 'path': 'project_old/utils/helpers.py'}]}]}

Provide a list of impacted dependencies and their significance.
Debug: Vector DB provided: False
Debug: Formatted prompt:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
None<|eot_id|><|start_header_id|>user<|end_header_id|>
Analyze the project context and determine the dependencies (edges) impacted by the following feature request:

Fea

/mnt/c/Blog/Building-LLM-from-Scratch-in-Python/.venv/lib/python3.10/site-packages/ibm_watsonx_ai/foundation_models/utils/utils.py:415: LifecycleWarning: Model 'meta-llama/llama-3-70b-instruct' is in deprecated state from 2024-12-02 until 2025-02-03. IDs of alternative models: None. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warnings.warn(


Debug: Querying without vector database...
Debug: Raw response from model: Based on the project context and the feature request, the impacted dependencies (edges) are:

1. **app.py -> logging module**: The feature request requires adding logging functionality to all major modules, including app.py. This means that app.py will now depend on the logging module, which is a Python standard library module.

Significance: High, as app.py is a major module in the project and logging functionality is a critical aspect of the feature request.

2. **helpers.py -> logging module**: Similar to app.py, helpers.py is also a major module in the project, and adding logging functionality to it will introduce a dependency on the logging module.

Significance: Medium to High, as helpers.py is a utility module that may be used by other parts of the project, and logging functionality is important for debugging and monitoring.

3. **requirements.txt -> logging module (indirectly)**: Although requirements.tx

In [ ]:
#3 impact_report

In [38]:
impact_report = impact_report_template.format(
    feature_request=user_request, project_context=project_context
)

In [40]:
impact_report_response = query_llm(user_input=impact_report)

Debug: Received prompt: Generate a detailed report on the impact of the following feature request on the project:

Feature Request: Add logging functionality to all major modules in the project

Project Context: requirements.txt: ['# Python dependencies\n', 'flask']{'project_name': 'project_old', 'structure': [{'type': 'file', 'name': 'app.py', 'path': 'project_old/app.py'}, {'type': 'file', 'name': 'requirements.txt', 'path': 'project_old/requirements.txt'}, {'type': 'directory', 'name': 'utils', 'path': 'project_old/utils', 'children': [{'type': 'file', 'name': 'helpers.py', 'path': 'project_old/utils/helpers.py'}]}]}{'project_name': 'project_old', 'structure': [{'type': 'file', 'name': 'app.py', 'path': 'project_old/app.py'}, {'type': 'file', 'name': 'requirements.txt', 'path': 'project_old/requirements.txt'}, {'type': 'directory', 'name': 'utils', 'path': 'project_old/utils', 'children': [{'type': 'file', 'name': 'helpers.py', 'path': 'project_old/utils/helpers.py'}]}]}{'project_na

/mnt/c/Blog/Building-LLM-from-Scratch-in-Python/.venv/lib/python3.10/site-packages/ibm_watsonx_ai/foundation_models/utils/utils.py:415: LifecycleWarning: Model 'meta-llama/llama-3-70b-instruct' is in deprecated state from 2024-12-02 until 2025-02-03. IDs of alternative models: None. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warnings.warn(


Debug: Querying without vector database...
Debug: Raw response from model: **Impact Report: Adding Logging Functionality to Major Modules**

**Introduction**

The proposed feature request aims to add logging functionality to all major modules in the project. This report assesses the impact of this request on the project, highlighting potential risks, suggestions for modular implementation, and expected changes.

**Project Overview**

The project, named "project_old," is a Flask-based application with a simple structure consisting of:

1. `app.py`: The main application file.
2. `requirements.txt`: The file listing Python dependencies.
3. `utils` directory: Containing a single file, `helpers.py`, with utility functions.

**Impact Analysis**

**Benefits**

1. **Improved Debugging**: Logging functionality will enable developers to track errors, warnings, and informational messages, making it easier to debug and troubleshoot issues.
2. **Enhanced Monitoring**: Logging will provide insights 

## Feature Analysis with VectorDB

In [14]:
vector_db = load_vector_database(persist_directory="./chroma_db")

In [ ]:
from src.vector_database.db_load import load_vector_db_by_collection
from src.models.llm_inference import query_llm

In [37]:
feature_nodes = feature_nodes_template.format(
    feature_request=user_request, project_context=project_context
)


In [38]:
feature_nodes

"Analyze the project context and identify the nodes (files or components) impacted by the following feature request:\n\nFeature Request: Add logging functionality to all major modules in the project\n\nProject Context: requirements.txt: ['# Python dependencies\\n', 'flask']{'project_name': 'project_old', 'structure': [{'type': 'file', 'name': 'app.py', 'path': 'project_old/app.py'}, {'type': 'file', 'name': 'requirements.txt', 'path': 'project_old/requirements.txt'}, {'type': 'directory', 'name': 'utils', 'path': 'project_old/utils', 'children': [{'type': 'file', 'name': 'helpers.py', 'path': 'project_old/utils/helpers.py'}]}]}\n\nProvide a list of impacted nodes and their roles in the project."

In [39]:
feature_nodes_response_db = query_llm(user_input=feature_nodes,vector_db=vector_db)

Debug: Received prompt: Analyze the project context and identify the nodes (files or components) impacted by the following feature request:

Feature Request: Add logging functionality to all major modules in the project

Project Context: requirements.txt: ['# Python dependencies\n', 'flask']{'project_name': 'project_old', 'structure': [{'type': 'file', 'name': 'app.py', 'path': 'project_old/app.py'}, {'type': 'file', 'name': 'requirements.txt', 'path': 'project_old/requirements.txt'}, {'type': 'directory', 'name': 'utils', 'path': 'project_old/utils', 'children': [{'type': 'file', 'name': 'helpers.py', 'path': 'project_old/utils/helpers.py'}]}]}

Provide a list of impacted nodes and their roles in the project.
Debug: Vector DB provided: True
Debug: Formatted prompt:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
None<|eot_id|><|start_header_id|>user<|end_header_id|>
Analyze the project context and identify the nodes (files or components) impacted by the following feature r

/mnt/c/Blog/Building-LLM-from-Scratch-in-Python/.venv/lib/python3.10/site-packages/ibm_watsonx_ai/foundation_models/utils/utils.py:415: LifecycleWarning: Model 'meta-llama/llama-3-70b-instruct' is in deprecated state from 2024-12-02 until 2025-02-03. IDs of alternative models: None. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warnings.warn(
Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3


Debug: Querying using vector database...
Debug: Raw response from vector DB: {'query': "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\nNone<|eot_id|><|start_header_id|>user<|end_header_id|>\nAnalyze the project context and identify the nodes (files or components) impacted by the following feature request:\n\nFeature Request: Add logging functionality to all major modules in the project\n\nProject Context: requirements.txt: ['# Python dependencies\\n', 'flask']{'project_name': 'project_old', 'structure': [{'type': 'file', 'name': 'app.py', 'path': 'project_old/app.py'}, {'type': 'file', 'name': 'requirements.txt', 'path': 'project_old/requirements.txt'}, {'type': 'directory', 'name': 'utils', 'path': 'project_old/utils', 'children': [{'type': 'file', 'name': 'helpers.py', 'path': 'project_old/utils/helpers.py'}]}]}\n\nProvide a list of impacted nodes and their roles in the project.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n", 'result': ' \n\nBased on the proj

In [40]:
feature_edges = feature_edges_template.format(
    feature_request=user_request, project_context=project_context
)

feature_edges_response_db = query_llm(user_input=feature_edges,vector_db=vector_db)

Debug: Received prompt: Analyze the project context and determine the dependencies (edges) impacted by the following feature request:

Feature Request: Add logging functionality to all major modules in the project

Project Context: requirements.txt: ['# Python dependencies\n', 'flask']{'project_name': 'project_old', 'structure': [{'type': 'file', 'name': 'app.py', 'path': 'project_old/app.py'}, {'type': 'file', 'name': 'requirements.txt', 'path': 'project_old/requirements.txt'}, {'type': 'directory', 'name': 'utils', 'path': 'project_old/utils', 'children': [{'type': 'file', 'name': 'helpers.py', 'path': 'project_old/utils/helpers.py'}]}]}

Provide a list of impacted dependencies and their significance.
Debug: Vector DB provided: True
Debug: Formatted prompt:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
None<|eot_id|><|start_header_id|>user<|end_header_id|>
Analyze the project context and determine the dependencies (edges) impacted by the following feature request:

Feat

/mnt/c/Blog/Building-LLM-from-Scratch-in-Python/.venv/lib/python3.10/site-packages/ibm_watsonx_ai/foundation_models/utils/utils.py:415: LifecycleWarning: Model 'meta-llama/llama-3-70b-instruct' is in deprecated state from 2024-12-02 until 2025-02-03. IDs of alternative models: None. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warnings.warn(
Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3


Debug: Querying using vector database...
Debug: Raw response from vector DB: {'query': "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\nNone<|eot_id|><|start_header_id|>user<|end_header_id|>\nAnalyze the project context and determine the dependencies (edges) impacted by the following feature request:\n\nFeature Request: Add logging functionality to all major modules in the project\n\nProject Context: requirements.txt: ['# Python dependencies\\n', 'flask']{'project_name': 'project_old', 'structure': [{'type': 'file', 'name': 'app.py', 'path': 'project_old/app.py'}, {'type': 'file', 'name': 'requirements.txt', 'path': 'project_old/requirements.txt'}, {'type': 'directory', 'name': 'utils', 'path': 'project_old/utils', 'children': [{'type': 'file', 'name': 'helpers.py', 'path': 'project_old/utils/helpers.py'}]}]}\n\nProvide a list of impacted dependencies and their significance.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n", 'result': " \n\nBased on the project con

In [41]:
impact_report = impact_report_template.format(
    feature_request=user_request, project_context=project_context
)

impact_report_response_db = query_llm(user_input=impact_report,vector_db=vector_db)

Debug: Received prompt: Generate a detailed report on the impact of the following feature request on the project:

Feature Request: Add logging functionality to all major modules in the project

Project Context: requirements.txt: ['# Python dependencies\n', 'flask']{'project_name': 'project_old', 'structure': [{'type': 'file', 'name': 'app.py', 'path': 'project_old/app.py'}, {'type': 'file', 'name': 'requirements.txt', 'path': 'project_old/requirements.txt'}, {'type': 'directory', 'name': 'utils', 'path': 'project_old/utils', 'children': [{'type': 'file', 'name': 'helpers.py', 'path': 'project_old/utils/helpers.py'}]}]}

Include risks, suggestions for modular implementation, and expected changes.
Debug: Vector DB provided: True
Debug: Formatted prompt:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
None<|eot_id|><|start_header_id|>user<|end_header_id|>
Generate a detailed report on the impact of the following feature request on the project:

Feature Request: Add logging fu

/mnt/c/Blog/Building-LLM-from-Scratch-in-Python/.venv/lib/python3.10/site-packages/ibm_watsonx_ai/foundation_models/utils/utils.py:415: LifecycleWarning: Model 'meta-llama/llama-3-70b-instruct' is in deprecated state from 2024-12-02 until 2025-02-03. IDs of alternative models: None. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warnings.warn(
Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3


Debug: Querying using vector database...
Debug: Raw response from vector DB: {'query': "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\nNone<|eot_id|><|start_header_id|>user<|end_header_id|>\nGenerate a detailed report on the impact of the following feature request on the project:\n\nFeature Request: Add logging functionality to all major modules in the project\n\nProject Context: requirements.txt: ['# Python dependencies\\n', 'flask']{'project_name': 'project_old', 'structure': [{'type': 'file', 'name': 'app.py', 'path': 'project_old/app.py'}, {'type': 'file', 'name': 'requirements.txt', 'path': 'project_old/requirements.txt'}, {'type': 'directory', 'name': 'utils', 'path': 'project_old/utils', 'children': [{'type': 'file', 'name': 'helpers.py', 'path': 'project_old/utils/helpers.py'}]}]}\n\nInclude risks, suggestions for modular implementation, and expected changes.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n", 'result': " \n\n**Impact Report: Adding Logging

In [ ]:
feature_nodes_response_db
feature_edges_response_db
impact_report_response_db

In [ ]:
analysis_results=feature_nodes_response_db+feature_nodes_response_db+impact_report_response_db

In [48]:
analysis_results=f"Nodes:\n{feature_nodes_response_db}\n\nEdges:\n{feature_nodes_response_db}\n\nReport:\n{impact_report_response_db}"

In [49]:
print(analysis_results)

Nodes:
Based on the project context, the nodes impacted by the feature request to add logging functionality to all major modules in the project are:

1. `app.py` (Role: Main application file)
2. `helpers.py` (Role: Utility module providing helper functions)

These nodes are impacted because they are the major modules in the project where logging functionality needs to be added. The `requirements.txt` file is not directly impacted as it only lists dependencies, and the `utils` directory is a container for the `helpers.py` module.

Edges:
Based on the project context, the nodes impacted by the feature request to add logging functionality to all major modules in the project are:

1. `app.py` (Role: Main application file)
2. `helpers.py` (Role: Utility module providing helper functions)

These nodes are impacted because they are the major modules in the project where logging functionality needs to be added. The `requirements.txt` file is not directly impacted as it only lists dependencies,

In [53]:
feature_nodes

"Analyze the project context and identify the nodes (files or components) impacted by the following feature request:\n\nFeature Request: Add logging functionality to all major modules in the project\n\nProject Context: requirements.txt: ['# Python dependencies\\n', 'flask']{'project_name': 'project_old', 'structure': [{'type': 'file', 'name': 'app.py', 'path': 'project_old/app.py'}, {'type': 'file', 'name': 'requirements.txt', 'path': 'project_old/requirements.txt'}, {'type': 'directory', 'name': 'utils', 'path': 'project_old/utils', 'children': [{'type': 'file', 'name': 'helpers.py', 'path': 'project_old/utils/helpers.py'}]}]}\n\nProvide a list of impacted nodes and their roles in the project."

In [54]:
# Step 1.5: Preprocessing
logger.info("Preprocessing results of feature analysis")
preprocessing_template = get_prompt_template("preprocessing_step")

In [55]:
preprocessing_template

'Using the results of the feature analysis, identify the specific files to be handled for the following feature request:\n\nFeature Request: {feature_request}\n\nFeature Analysis Results:\n{analysis_results}\n\nFor each impacted file:\n- Provide the file path.\n- Specify the task to be performed on the file (e.g., modify, add functionality).\n\nFor any new file required:\n- Specify the file path.\n- Provide a brief description of its purpose.\n- Suggest the initial content or structure the file should include.\n\nEnsure the output clearly separates existing files and new files to be created.'

In [56]:


# Include the results of feature analysis (nodes and edges)
preprocessing_prompt = preprocessing_template.format(
    feature_request=user_request,
    analysis_results=f"Nodes:\n{feature_nodes_response_db}\n\nEdges:\n{feature_nodes_response_db}\n\nReport:\n{impact_report_response_db}"
)

In [59]:
print(preprocessing_prompt)


Using the results of the feature analysis, identify the specific files to be handled for the following feature request:

Feature Request: Add logging functionality to all major modules in the project

Feature Analysis Results:
Nodes:
Based on the project context, the nodes impacted by the feature request to add logging functionality to all major modules in the project are:

1. `app.py` (Role: Main application file)
2. `helpers.py` (Role: Utility module providing helper functions)

These nodes are impacted because they are the major modules in the project where logging functionality needs to be added. The `requirements.txt` file is not directly impacted as it only lists dependencies, and the `utils` directory is a container for the `helpers.py` module.

Edges:
Based on the project context, the nodes impacted by the feature request to add logging functionality to all major modules in the project are:

1. `app.py` (Role: Main application file)
2. `helpers.py` (Role: Utility module providi

In [60]:
preprocessing_response_db = query_llm(user_input=preprocessing_prompt,vector_db=vector_db)

Debug: Received prompt: Using the results of the feature analysis, identify the specific files to be handled for the following feature request:

Feature Request: Add logging functionality to all major modules in the project

Feature Analysis Results:
Nodes:
Based on the project context, the nodes impacted by the feature request to add logging functionality to all major modules in the project are:

1. `app.py` (Role: Main application file)
2. `helpers.py` (Role: Utility module providing helper functions)

These nodes are impacted because they are the major modules in the project where logging functionality needs to be added. The `requirements.txt` file is not directly impacted as it only lists dependencies, and the `utils` directory is a container for the `helpers.py` module.

Edges:
Based on the project context, the nodes impacted by the feature request to add logging functionality to all major modules in the project are:

1. `app.py` (Role: Main application file)
2. `helpers.py` (Role

/mnt/c/Blog/Building-LLM-from-Scratch-in-Python/.venv/lib/python3.10/site-packages/ibm_watsonx_ai/foundation_models/utils/utils.py:415: LifecycleWarning: Model 'meta-llama/llama-3-70b-instruct' is in deprecated state from 2024-12-02 until 2025-02-03. IDs of alternative models: None. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warnings.warn(


Debug: Querying using vector database...


Number of requested results 5 is greater than number of elements in index 4, updating n_results = 4


Debug: Raw response from vector DB: {'query': "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\nNone<|eot_id|><|start_header_id|>user<|end_header_id|>\nUsing the results of the feature analysis, identify the specific files to be handled for the following feature request:\n\nFeature Request: Add logging functionality to all major modules in the project\n\nFeature Analysis Results:\nNodes:\nBased on the project context, the nodes impacted by the feature request to add logging functionality to all major modules in the project are:\n\n1. `app.py` (Role: Main application file)\n2. `helpers.py` (Role: Utility module providing helper functions)\n\nThese nodes are impacted because they are the major modules in the project where logging functionality needs to be added. The `requirements.txt` file is not directly impacted as it only lists dependencies, and the `utils` directory is a container for the `helpers.py` module.\n\nEdges:\nBased on the project context, the nodes impacted by 

In [61]:
preprocessing_response_db

'**Impacted Files:**\n\n1. **File Path:** `../project_old/app.py`\n**Task:** Modify the file to integrate with the new logging module and utilize the logging functionality.\n\n2. **File Path:** `../project_old/utils/helpers.py`\n**Task:** Modify the file to integrate with the new logging module and utilize the logging functionality.\n\n3. **File Path:** `../project_old/requirements.txt`\n**Task:** Update the file to include the chosen logging library.\n\n**New Files:**\n\n1. **File Path:** `../project_old/logger.py`\n**Purpose:** Encapsulate the logging functionality.\n**Initial Content/Structure:** The file should include the necessary imports, logging configuration, and logging functions.\n\n2. **File Path:** `../project_old/config/logging_config.py` (or similar)\n**Purpose:** Manage logging settings, such as log levels, output formats, and destinations.\n**Initial Content/Structure:** The file should include configuration options for logging, such as log levels, output formats, and 

In [46]:
from src.vector_database.db_load import load_vector_db_by_collection
from src.models.llm_inference import query_llm

# Define the persistent directory and collection name
persist_directory = "./notebooks/chroma_db"
collection_name = "my_vector_collection"

# Load the vector database
vector_db = load_vector_db_by_collection(collection_name, persist_directory)

# Query the vector database using the `query_llm` function
response = query_llm(user_input=prompt, vector_db=vector_db)

# Print the response
print("Generated Response:")
print(response)

NameError: name 'prompt' is not defined

In [ ]:

# Step 2: Task Generation and Agent Creation
logger.info("Generating tasks and creating agents")
task_generation_template = get_prompt_template("task_generation")
agent_creation_template = get_prompt_template("agent_creation")
weight_translation_template = get_prompt_template("weight_translation")

tasks = task_generation_template.format(
    feature_request=user_request, project_context=project_context
)
agents = agent_creation_template.format(tasks=tasks)
weights = weight_translation_template.format(project_elements=project_context)

# Step 3: Task Assignment
logger.info("Assigning tasks to agents")
task_assignment_template = get_prompt_template("task_assignment")
task_assignment = task_assignment_template.format(
    tasks=tasks, agent_details=agents
)

# Step 4: Task Optimization
logger.info("Optimizing task allocation")
task_optimization_template = get_prompt_template("task_optimization")
optimized_tasks = task_optimization_template.format(
    task_assignments=task_assignment
)

# Step 5: Consistency Validation
logger.info("Validating consistency of the project tree")
consistency_validation_template = get_prompt_template("consistency_validation")
validated_tree = consistency_validation_template.format(project_tree=project_context)


In [ ]:



relevant_docs = query_vector_database(vector_db, feature_request)
logger.info(f"Relevant documents for the feature request: {[doc.metadata['source'] for doc in relevant_docs]}")

# Step 4: Generate new project structure
logger.info("Generating the updated project structure")
if os.path.exists(new_project_path):
    shutil.rmtree(new_project_path)
shutil.copytree(old_project_path, new_project_path)

# Step 5: Integrate new features
logger.info("Integrating new features into the project")
for doc in relevant_docs:
    integrate_features([doc.metadata["source"]], f"# Feature: {prompt}\n{doc.page_content}")

# Step 6: Generate additional files or modify existing ones
feature_instructions = [
    {"file": os.path.relpath(doc.metadata["source"], old_project_path), "content": f"# Feature: {prompt}\n{doc.page_content}"}
    for doc in relevant_docs
]
generate_project(old_project_path, new_project_path, feature_instructions)

logger.info("Feature integration completed")
print(f"Project updated with the feature: {prompt}")
print(f"Updated project saved in: {new_project_path}")


In [ ]:
def main(prompt):
    logger.info("Starting the Factory Feature program")

    # Define paths
    old_project_path = "project_old"
    new_project_path = "project_new"

    # Step 1: Parse the old project
    logger.info("Parsing the project structure")
    project_data = parse_project(old_project_path)
    vector_db = build_vector_database(project_data)
    logger.info("Vector database created for the project")

    # Step 2: Resolve dependencies
    logger.info("Resolving project dependencies")
    dependencies = resolve_dependencies(old_project_path)
    logger.info(f"Dependencies found: {dependencies.keys()}")

    # Step 3: Map features to project components
    logger.info("Mapping feature request to project components")
    feature_template = get_prompt_template()
    project_context = "\n".join([f"{key}: {value}" for key, value in dependencies.items()])
    feature_request = feature_template.format(feature_request=prompt, project_context=project_context)
    relevant_docs = query_vector_database(vector_db, feature_request)
    logger.info(f"Relevant documents for the feature request: {[doc.metadata['source'] for doc in relevant_docs]}")

    # Step 4: Generate new project structure
    logger.info("Generating the updated project structure")
    if os.path.exists(new_project_path):
        shutil.rmtree(new_project_path)
    shutil.copytree(old_project_path, new_project_path)

    # Step 5: Integrate new features
    logger.info("Integrating new features into the project")
    for doc in relevant_docs:
        integrate_features([doc.metadata["source"]], f"# Feature: {prompt}\n{doc.page_content}")

    # Step 6: Generate additional files or modify existing ones
    feature_instructions = [
        {"file": os.path.relpath(doc.metadata["source"], old_project_path), "content": f"# Feature: {prompt}\n{doc.page_content}"}
        for doc in relevant_docs
    ]
    generate_project(old_project_path, new_project_path, feature_instructions)

    logger.info("Feature integration completed")
    print(f"Project updated with the feature: {prompt}")
    print(f"Updated project saved in: {new_project_path}")


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Factory Feature Program")
    parser.add_argument("--prompt", required=True, help="Feature request prompt")
    args = parser.parse_args()

    try:
        main(args.prompt)
    except Exception as e:
        logger.error(f"An error occurred: {e}")
        raise
